# ACC with stop&go

In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [3]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*', transform = carla.Transform()):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += transform.location.x
    spawn_point.location.y += transform.location.y
    spawn_point.location.z += transform.location.z
    spawn_point.rotation.pitch += transform.rotation.pitch
    spawn_point.rotation.roll += transform.rotation.roll
    spawn_point.rotation.yaw += transform.rotation.yaw
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    #camera_bp.set_attribute('fov', '120')
    #camera_bp.set_attribute('sensor_tick', '0')
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), range=50, points_per_second=10):
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', '30')
    radar_bp.set_attribute('vertical_fov', '30')
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar


## Sensors

Sensors are another type of actors, usually spawned attached to a vehicle, designed to retrieve data from the world. The type of data depends on the sensor, but it can range from RGB images, LiDAR scans or even collision information.

Sensors are divided into two main categories:

- **Regular sensors**: these sensors retrieve data at a fixed rate, usually every tick (e.g RGB cameras).
- **Trigger sensors**: these sensors only retrieve data when a certain condition is met (e.g collision sensors).

Sensors details can be found in the [CARLA documentation](https://carla.readthedocs.io/en/latest/ref_sensors/).

![Sensors](img/sensors.jpg)

### Live camera feed

We can also visualize the camera feed in real-time. This is useful for debugging and understanding what the camera sees.

We can use the `opencv` library to create a window and display the camera feed.

In [7]:

pygame.init()
# focus on the pygame window to get the key pressed
pygame.display.set_mode((400, 300))

ego_vehicle = spawn_vehicle()
vehicle2 = spawn_vehicle(transform=carla.Transform(carla.Location(x=50)))
camera = spawn_camera(attach_to=ego_vehicle, transform=carla.Transform(carla.Location(x=-6, z=5), carla.Rotation(pitch=-30)))

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

#camera.listen(lambda image: camera_callback(image))

#vehicle.set_autopilot(True)
#control = carla.VehicleControl(throttle=1000)
#vehicle.apply_control(control)

#cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True
moving_forward = False

try:
    while running:
        for event in pygame.event.get():
            print("py")
            if event.type == pygame.KEYDOWN:
                key = event.key
                if key == pygame.QUIT or key == pygame.K_ESCAPE:
                    running = False
                elif key == pygame.K_w:
                    moving_forward = True
                elif key == pygame.K_s:
                    moving_forward = False

        if moving_forward:
            control = carla.VehicleControl(throttle=0.5)
        else:
            control = carla.VehicleControl(throttle=0.0)

        ego_vehicle.apply_control(control)
        world.tick()
        
        pygame.display.flip()


        print("Actor transform: ", ego_vehicle.get_transform())
        print("Actor forward vector (direction):", ego_vehicle.get_transform().get_forward_vector())
        print("Actor velocity: ", ego_vehicle.get_velocity(),", ",ego_vehicle.get_velocity().length(),"m/s, ",ego_vehicle.get_velocity().length()*3.6,"km/h")
        #cv2.imshow('RGB Camera', video_output)
        move_spectator_to(ego_vehicle.get_transform())
        
finally:
    pygame.quit()
    #cv2.destroyAllWindows()
    camera.destroy()
    ego_vehicle.destroy()

py
py
py
py
Actor transform:  Transform(Location(x=-64.644371, y=24.470930, z=-0.007605), Rotation(pitch=0.071225, yaw=0.159189, roll=-0.010925))
Actor forward vector (direction): Vector3D(x=0.999995, y=0.002778, z=0.001243)
Actor velocity:  Vector3D(x=0.000233, y=0.000986, z=0.024630) ,  0.02465091645717621 m/s,  0.08874329924583435 km/h
Actor transform:  Transform(Location(x=-64.644409, y=24.470932, z=-0.006502), Rotation(pitch=0.066335, yaw=0.159198, roll=-0.004181))
Actor forward vector (direction): Vector3D(x=0.999995, y=0.002779, z=0.001158)
Actor velocity:  Vector3D(x=0.000091, y=0.000426, z=0.002475) ,  0.002512633800506592 m/s,  0.00904548168182373 km/h
Actor transform:  Transform(Location(x=-64.644424, y=24.470934, z=-0.006668), Rotation(pitch=0.064579, yaw=0.159201, roll=-0.001434))
Actor forward vector (direction): Vector3D(x=0.999995, y=0.002779, z=0.001127)
Actor velocity:  Vector3D(x=0.000029, y=0.000166, z=-0.003807) ,  0.003810860915109515 m/s,  0.013719099294394256 km

In [ ]:
camera_transforms = [
    (carla.Transform(carla.Location(x=1.5, z=2.4)), (600, 300)),  # Front camera
    (carla.Transform(carla.Location(x=-0.5, y=-0.9, z=2.4), carla.Rotation(yaw=-135)), (200, 400)),  # Left side camera
    (carla.Transform(carla.Location(x=-0.5, y=0.9, z=2.4), carla.Rotation(yaw=135)), (200, 400)),  # Right side camera
    (carla.Transform(carla.Location(x=-1.5, z=2.4), carla.Rotation(yaw=180)), (600, 300))  # Rear camera
]

ego_vehicle = spawn_vehicle()
#control = carla.VehicleControl(throttle=0.5)
#vehicle.apply_control(control)
#world.tick()
ego_vehicle.set_autopilot(True)


# Spawn cameras and attach to vehicle
cameras = []
video_outputs = [np.zeros((600, 800, 4), dtype=np.uint8) for _ in range(4)]

def create_camera_callback(index):
    def camera_callback(image):
        global video_outputs
        video_outputs[index] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    return camera_callback

for i, transform in enumerate(camera_transforms):
    camera = spawn_camera(attach_to=ego_vehicle, transform=transform[0], width=transform[1][0], height=transform[1][1])
    camera.listen(create_camera_callback(i))
    cameras.append(camera)

cv2.namedWindow('Front Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Left Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Right Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Rear Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('Front Camera', video_outputs[0])
        cv2.imshow('Left Side Camera', video_outputs[1])
        cv2.imshow('Right Side Camera', video_outputs[2])
        cv2.imshow('Rear Camera', video_outputs[3])
finally:
    cv2.destroyAllWindows()
    for camera in cameras:
        camera.destroy()
    ego_vehicle.destroy()

In [4]:
ego_vehicle = spawn_vehicle()

gps_blueprint = world.get_blueprint_library().find('sensor.other.gnss')
gps_sensor = world.spawn_actor(gps_blueprint, carla.Transform(), attach_to=ego_vehicle)
print(carla.Transform())

ROAD_X_MIN = -110
LONGITUDE_MAX = 0.001

latest_gps = {"latitude": 0.0, "longitude": 0.0}

def gps_callback(gps_data):
    global latest_gps
    latest_gps["latitude"] = gps_data.latitude
    latest_gps["longitude"] = gps_data.longitude

gps_sensor.listen(gps_callback)

try:
    while True:
        current_x = latest_gps["longitude"]
        print(f"Longitude: {current_x}", end='\r')

        if current_x > LONGITUDE_MAX:
            new_location = carla.Location(x=ROAD_X_MIN, y=ego_vehicle.get_transform().location.y)
            ego_vehicle.set_transform(carla.Transform(new_location, ego_vehicle.get_transform().rotation))

        control = carla.VehicleControl(throttle=0.5)
        ego_vehicle.apply_control(control)

        world.tick()
except KeyboardInterrupt:
    pass
finally:
    gps_sensor.destroy()
    ego_vehicle.destroy()

Transform(Location(x=0.000000, y=0.000000, z=0.000000), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))


In [4]:
#TTC = (distance / relative_velocity) if relative_velocity != 0 else 9999
def type_detection(data) -> carla.RadarDetection:
    return data
    

# Simple callback function to print the number of detections
def handle_measurement(data: carla.RadarMeasurement, vehicle: carla.Vehicle):
    print(data.get_detection_count())
    for measurement in data:
        measurement = type_detection(measurement)
        # If average ttc is less than 1 second, apply brake
        if measurement.ttc < 1:
            control = carla.VehicleControl(brake=1.0)
            vehicle.apply_control(control)
        

In [5]:
# Remove all actors
for actor in world.get_actors():
    actor.destroy()
    
# Spawn a ego vehicle and attach a radar sensor to it 
ego_vehicle = spawn_vehicle(spawn_index=0)
radar = spawn_radar(attach_to=ego_vehicle)
time.sleep(2)
ego_start = ego_vehicle.get_transform()
# and another vehicle to detect, little ahead of the ego vehicle
another_vehicle = spawn_vehicle(transform=carla.Transform(carla.Location(x=50)))

#ego_vehicle.set_autopilot(True)

# Move the spectator to the ego vehicle
radar.listen(lambda data: handle_measurement(data, ego_vehicle))
try:
    while True:
        move_spectator_to(ego_vehicle.get_transform())
        world.tick()
except KeyboardInterrupt:
    pass
finally:
    radar.destroy()
    ego_vehicle.destroy()
    another_vehicle.destroy()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


AttributeError: 'RadarDetection' object has no attribute 'ttc'

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000